In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# define the dataset path
TRAIN_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/csv/sequential_feature_selection_features_updated_train_new.csv"

In [ ]:
!pip install lifelines torchtuples

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 2.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=4482eb0188bd2524a6de451e9859fea38d8acaeef64461c120b8e0b4770a68f5
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [ ]:
import pandas as pd
import numpy as np
# read the data
data = pd.read_csv(TRAIN_PATH)
data.head()

,C20orf22,ADCK2,ARL5B,BMPR1A,BMS1P1,FRA10AC1,OGFOD3,SPDL1,CCNJ,CD81,...,SMCP,SNORA36B,UNC93B1,WIPF3,ZNF22,ZNF248,FLJ23404,ZNF676,Disease Free (Months),DFS_STATUS_ENCODED
0,1.1084,0.1381,-1.3015,-0.3298,-0.8790,-0.2001,0.2880,-0.6509,-0.3501,1.3306,...,-1.2819,-47.5219,0.3042,-0.8972,-0.7499,-1.2925,-0.3176,-1.1132,8.41,1.0
1,-0.1005,0.5212,-0.2573,-0.4503,-0.2721,0.5470,-0.0534,-1.0111,-0.0342,-0.3876,...,-1.2819,-47.5219,0.4726,1.0643,-1.4436,0.4544,0.5857,-0.5119,2.56,1.0
2,0.3004,-0.9082,-0.1168,-0.3587,0.8405,-0.7239,-0.1667,0.8176,0.9477,-0.2253,...,-1.2819,-47.5219,0.5460,-0.8035,0.0617,-0.0401,0.2981,-0.5328,1.22,0.0
3,-0.4013,0.7237,-0.7190,-0.3022,0.4796,0.0019,-0.3718,-0.0256,-0.8363,-0.1055,...,-1.2819,-47.5219,0.9282,1.8169,-0.8905,-0.2346,0.3443,-0.7956,13.40,1.0
4,-0.3392,-0.0252,1.1848,0.8085,0.5990,1.4794,-1.6423,0.4705,1.6616,-1.3079,...,-1.2819,-47.5219,-0.2614,-0.1870,0.2724,1.1534,1.3512,-0.8140,3.19,1.0


In [ ]:
#Drop cols based on convergence

columns_to_drop = ['KRTAP9-4', 'OR2T2', 'OR4F17', 'SNORA36B']
data.drop(columns_to_drop, axis=1, inplace = True)

Creating deep survival model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtuples import Model
import torchtuples as tt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index

In [ ]:
import torch
import torch.nn as nn

# Define the DeepSurv model with additional layers and dropout
class DeepSurv(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout_prob=0.2):
        super(DeepSurv, self).__init__()
        self.layers = nn.ModuleList()
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)

        # Create hidden layers with dropout
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            self.layers.append(nn.Linear(prev_dim, hidden_dim))
            prev_dim = hidden_dim

        # Output layer
        self.output_layer = nn.Linear(hidden_dims[-1], 1)

    def forward(self, x):
        # Pass through each hidden layer with ReLU activation and dropout
        for layer in self.layers:
            x = self.relu(layer(x))
            x = self.dropout(x)  # Apply dropout after activation
        return self.output_layer(x)

# Function to get the loss (Negative Log-Likelihood)
def nll_loss(preds, durations, events):
    # Cox proportional hazards loss function
    hazards = preds.squeeze()
    log_hazards = torch.log(torch.cumsum(torch.exp(hazards), dim=0))
    uncensored_loss = hazards - log_hazards
    censored_loss = uncensored_loss * events
    return -torch.sum(censored_loss)


Separate x and y

In [ ]:
DEPENDENT_COLUMNS = ['Disease Free (Months)', 'DFS_STATUS_ENCODED']
GENES = [cols for cols in list(data.columns) if cols not in DEPENDENT_COLUMNS]
#prepare the dataset
X = data[GENES]
y = data[DEPENDENT_COLUMNS]

In [ ]:
X.shape

(73, 46)

Cross validation

In [ ]:
# #import numpy as np
# from sklearn.model_selection import KFold
# from tqdm.notebook import tqdm

# #change epoch and learning rate and record mean c-index value in hyperparameter sheet

# #single layer
# HIDDEN_DIMENSION = 64

# #multiple layers
# #HIDDEN_DIMENSION = [64, 32]
# #HIDDEN_DIMENSION = [128, 64]
# #HIDDEN_DIMENSION = [128, 64, 32]
# #HIDDEN_DIMENSION = [64, 32, 16]
# #HIDDEN_DIMENSION = [128, 64, 32]
# NO_EPOCHS = 70
# LEARNING_RATE = 0.001

# conco_index = []

# # KFold Cross-validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# for train_index, valid_index in kf.split(X):
#     X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
#     y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

#     # Convert data to tensors
#     X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32)
#     y_train_duration = torch.tensor(y_train_fold['Disease Free (Months)'].values, dtype=torch.float32)
#     y_train_event = torch.tensor(y_train_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#     X_test_tensor = torch.tensor(X_valid_fold.values, dtype=torch.float32)
#     y_test_duration = torch.tensor(y_valid_fold['Disease Free (Months)'].values, dtype=torch.float32)
#     y_test_event = torch.tensor(y_valid_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#     # define the model and the optimizer
#     input_dim = X.shape[1]
#     model = DeepSurv(input_dim, HIDDEN_DIMENSION)
#     optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE)

#     # train the model
#     for epoch in tqdm(range(NO_EPOCHS)):
#         model.train()
#         optimizer.zero_grad()

#         # Forward pass
#         preds = model(X_train_tensor)

#         # Compute the loss
#         loss = nll_loss(preds, y_train_duration, y_train_event)
#         loss.backward()
#         optimizer.step()

#     # Evaluate the model
#     model.eval()
#     with torch.no_grad():
#         test_preds = model(X_test_tensor)
#         c_index = concordance_index(y_test_duration, -test_preds.numpy(), y_test_event)
#         conco_index.append(c_index)
#         print(f'Concordance Index: {c_index:.4f}')

# # print evaluations
# print("Cross Validation C-Indexes: {}".format(conco_index))
# print("Mean C-Index: {}".format(np.array(conco_index).mean()))

In [ ]:
import os
# Save the entire model to a file
SAVE_MODEL_PATH = "/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/DeepSurvModel"
MODEL_NAME = "sequential_feature_search_DeepSurv_try5.pth"
# save the entire model with architecture
# torch.save(model, os.path.join(SAVE_MODEL_PATH, MODEL_NAME))

In [ ]:
import pandas as pd
from sklearn.model_selection import KFold

# Define parameters for experimentation
epochs_list = [50, 60, 70, 80, 90, 100]
learning_rates = [0.00001, 0.00005, 0.0001, 0.0005, 0.001]
#hidden_layers = [256, 128, 64, 32]
hidden_layers = [128, 64, 32]
#hidden_layers = [128, 64]
#hidden_layers = [64, 32]


# List to store results
results = []

# Variable to track the best model
best_model_state = None
best_c_index = -np.inf  # Initialize with negative infinity
best_params = None  # To store parameters of the best model

# Experiment loop
for epochs in epochs_list:
    for lr in learning_rates:
        conco_index = []  # To store c-index for cross-validation

        # KFold Cross-validation
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        # KFold Cross-validation
        for train_index, valid_index in kf.split(X):
            X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
            y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

            # Convert data to tensors
            X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32)
            y_train_duration = torch.tensor(y_train_fold['Disease Free (Months)'].values, dtype=torch.float32)
            y_train_event = torch.tensor(y_train_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

            X_test_tensor = torch.tensor(X_valid_fold.values, dtype=torch.float32)
            y_test_duration = torch.tensor(y_valid_fold['Disease Free (Months)'].values, dtype=torch.float32)
            y_test_event = torch.tensor(y_valid_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

            # Define the model and optimizer
            input_dim = X.shape[1]
            model = DeepSurv(input_dim, hidden_layers)
            optimizer = optim.Adam(model.parameters(), lr=lr)

            # Train the model
            for epoch in range(epochs):
                model.train()
                optimizer.zero_grad()

                # Forward pass
                preds = model(X_train_tensor)

                # Compute the loss
                loss = nll_loss(preds, y_train_duration, y_train_event)
                loss.backward()
                optimizer.step()

            # Evaluate the model
            model.eval()
            with torch.no_grad():
                test_preds = model(X_test_tensor)
                c_index = concordance_index(y_test_duration, -test_preds.numpy(), y_test_event)
                conco_index.append(c_index)

        # Record the mean concordance index for this configuration
        mean_c_index = np.mean(conco_index)
        results.append({
            'Epochs': epochs,
            'Learning Rate': lr,
            'Mean C-Index': mean_c_index
        })
        print(f'Epochs: {epochs}, Learning Rate: {lr}, Mean C-Index: {mean_c_index:.4f}')

        # Update the best model if this configuration is the best so far
        if mean_c_index > best_c_index:
            best_c_index = mean_c_index
            best_model_state = model.state_dict()  # Save the model's state dictionary
            best_params = {'Epochs': epochs, 'Learning Rate': lr}

# Save the best model to a .pth file
torch.save(model, os.path.join(SAVE_MODEL_PATH, MODEL_NAME))
print(f"Best model saved with C-Index: {best_c_index:.4f} and parameters: {best_params}")

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/DeepSurv/deepsurv_SFS_results_try5.csv', index=False)
print("Experimentation complete. Results saved to 'deepsurv_SFS_results_try5.csv'.")


Epochs: 50, Learning Rate: 1e-05, Mean C-Index: 0.5544
Epochs: 50, Learning Rate: 5e-05, Mean C-Index: 0.5712
Epochs: 50, Learning Rate: 0.0001, Mean C-Index: 0.6517
Epochs: 50, Learning Rate: 0.0005, Mean C-Index: 0.6780
Epochs: 50, Learning Rate: 0.001, Mean C-Index: 0.6408
Epochs: 60, Learning Rate: 1e-05, Mean C-Index: 0.4770
Epochs: 60, Learning Rate: 5e-05, Mean C-Index: 0.6312
Epochs: 60, Learning Rate: 0.0001, Mean C-Index: 0.6980
Epochs: 60, Learning Rate: 0.0005, Mean C-Index: 0.6428
Epochs: 60, Learning Rate: 0.001, Mean C-Index: 0.6264
Epochs: 70, Learning Rate: 1e-05, Mean C-Index: 0.5760
Epochs: 70, Learning Rate: 5e-05, Mean C-Index: 0.5659
Epochs: 70, Learning Rate: 0.0001, Mean C-Index: 0.6721
Epochs: 70, Learning Rate: 0.0005, Mean C-Index: 0.6350
Epochs: 70, Learning Rate: 0.001, Mean C-Index: 0.6275
Epochs: 80, Learning Rate: 1e-05, Mean C-Index: 0.5637
Epochs: 80, Learning Rate: 5e-05, Mean C-Index: 0.6892
Epochs: 80, Learning Rate: 0.0001, Mean C-Index: 0.6806
Epo

In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.model_selection import KFold
# from lifelines.utils import concordance_index

# # Define parameters for experimentation
# epochs_list = [50, 60, 70, 80, 90, 100]
# learning_rates = [0.00001, 0.00005, 0.0001, 0.0005, 0.001]
# hidden_layers_list = [
#     [128, 64, 32],
#     [64, 32],
#     [64]
# ]
# dropout_rates = [0.2, 0.3, 0.4, 0.5]

# # List to store results
# results = []

# # Variable to track the best model
# best_model = None
# best_model_state = None
# best_c_index = -np.inf  # Initialize with negative infinity
# best_params = None  # To store parameters of the best model

# # Experiment loop
# for hidden_layers in hidden_layers_list:
#     for dropout_rate in dropout_rates:
#         for epochs in epochs_list:
#             for lr in learning_rates:
#                 conco_index = []  # To store c-index for cross-validation

#                 # KFold Cross-validation
#                 kf = KFold(n_splits=5, shuffle=True, random_state=42)

#                 for train_index, valid_index in kf.split(X):
#                     X_train_fold, X_valid_fold = X.iloc[train_index], X.iloc[valid_index]
#                     y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

#                     # Convert data to tensors
#                     X_train_tensor = torch.tensor(X_train_fold.values, dtype=torch.float32)
#                     y_train_duration = torch.tensor(y_train_fold['Disease Free (Months)'].values, dtype=torch.float32)
#                     y_train_event = torch.tensor(y_train_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#                     X_test_tensor = torch.tensor(X_valid_fold.values, dtype=torch.float32)
#                     y_test_duration = torch.tensor(y_valid_fold['Disease Free (Months)'].values, dtype=torch.float32)
#                     y_test_event = torch.tensor(y_valid_fold['DFS_STATUS_ENCODED'].values, dtype=torch.float32)

#                     # Define the model and optimizer
#                     input_dim = X.shape[1]
#                     model = DeepSurv(input_dim, hidden_layers, dropout_rate)
#                     optimizer = optim.Adam(model.parameters(), lr=lr)

#                     # Train the model
#                     for epoch in range(epochs):
#                         model.train()
#                         optimizer.zero_grad()

#                         # Forward pass
#                         preds = model(X_train_tensor)

#                         # Compute the loss
#                         loss = nll_loss(preds, y_train_duration, y_train_event)
#                         loss.backward()
#                         optimizer.step()

#                     # Evaluate the model
#                     model.eval()
#                     with torch.no_grad():
#                         test_preds = model(X_test_tensor)
#                         c_index = concordance_index(y_test_duration, -test_preds.numpy(), y_test_event)
#                         conco_index.append(c_index)

#                 # Record the mean concordance index for this configuration
#                 if conco_index:  # Ensure the list is not empty
#                     mean_c_index = np.mean(conco_index)
#                     results.append({
#                         'Hidden Layers': hidden_layers,
#                         'Dropout Rate': dropout_rate,
#                         'Epochs': epochs,
#                         'Learning Rate': lr,
#                         'Mean C-Index': mean_c_index
#                     })
#                     print(f'Hidden Layers: {hidden_layers}, Dropout Rate: {dropout_rate}, Epochs: {epochs}, '
#                           f'Learning Rate: {lr}, Mean C-Index: {mean_c_index:.4f}')

#                     # Update the best model if this configuration is the best so far
#                     if mean_c_index > best_c_index:
#                         best_c_index = mean_c_index
#                         best_model_state = model.state_dict()  # Save the model's state dictionary
#                         best_model = model
#                         best_params = {
#                             'Hidden Layers': hidden_layers,
#                             'Dropout Rate': dropout_rate,
#                             'Epochs': epochs,
#                             'Learning Rate': lr
#                         }

# # Save the best model to a .pth file
# torch.save(best_model, os.path.join(SAVE_MODEL_PATH, MODEL_NAME))
# print(f"Best model saved with C-Index: {best_c_index:.4f} and parameters: {best_params}")

# # Convert results to DataFrame and save to CSV
# results_df = pd.DataFrame(results)
# results_df.to_csv('/content/drive/Shareddrives/1:1_Krish_Nachnani/2024/Glioblastoma Multiforme -genome data/DeepSurv/deepsurv_SFS_results.csv', index=False)
# print("Experimentation complete. Results saved to 'deepsurv_SFS_results.csv'.")
